In [1]:
import requests
import pandas as pd
import json

In [2]:
from apiclient.discovery import build
from csv import writer
from urllib.parse import urlparse, parse_qs

In [3]:
def get_keys(filename):
    with open(filename) as f:
        key = f.readline()
    DEVELOPER_KEY = key
    YOUTUBE_API_SERVICE_NAME = "youtube"
    YOUTUBE_API_VERSION = "v3"
    return {'key': key, 'name': 'youtube', 'version': 'v3'}

In [4]:
def build_service(filename):
    with open(filename) as f:
        key = f.readline()

    YOUTUBE_API_SERVICE_NAME = "youtube"
    YOUTUBE_API_VERSION = "v3"
    return build(YOUTUBE_API_SERVICE_NAME,
                 YOUTUBE_API_VERSION,
                 developerKey=key)

In [5]:
def get_id(url):
    u_pars = urlparse(url)
    quer_v = parse_qs(u_pars.query).get('v')
    if quer_v:
        return quer_v[0]
    pth = u_pars.path.split('/')
    if pth:
        return pth[-1]

In [6]:
def save_to_csv(output_dict, filename):
    output_df = pd.DataFrame(output_dict, columns = output_dict.keys())
    output_df.to_csv(f'{filename}.csv')

In [40]:
def comments_helper(video_ID, api_key, service, filename):
    comments, commentsId, likesCount, viewerRating, authors = [], [], [], [], []
    
    response = service.commentThreads().list(
        part="snippet", 
        videoId = video_ID, 
        textFormat="plainText").execute()
    
    page = 0
    
    while page < 2:
        print(f'Page no: {page}')
        page += 1
        index = 0
        
        for item in response['items']:
            print(f"comment {index}")
            
            index += 1

            comment = item["snippet"]["topLevelComment"]
            
            author = comment["snippet"]["authorDisplayName"]
            text = comment["snippet"]["textDisplay"]
            comment_id = item['snippet']['topLevelComment']['id']
            like_count = item['snippet']['topLevelComment']['snippet']['likeCount']
            viewer_rating = item['snippet']['topLevelComment']['snippet']['viewerRating']

            # append to corresponding list
            comments.append(text)
            commentsId.append(comment_id)
            likesCount.append(like_count)
            viewerRating.append(viewer_rating)
            authors.append(author)

            with open(f'{csv_filename}.csv', 'a+') as f:
                csv_writer = writer(f)
                csv_writer.writerow([text, comment_id, like_count, viewer_rating, author])

        
        # check if there's a next page
        if 'nextPageToken' in response:
            response = service.commentThreads().list(
                part="snippet", 
                videoId = video_ID, 
                textFormat="plainText",
                pageToken=response['nextPageToken']
            ).execute()
        else:
            break

    return {
        'Comment': comments,
        'Author' : authors,
        'Comment ID' : commentsId,
        'Like Count' : likesCount,
        'Viewer Rating': viewerRating
    }
    

In [41]:
def get_comments(video_url, api_key, order = 'time', 
                   part = 'snippet', maxResults = 100, 
                   csv_filename = 'video.csv' ):
    
    yt_service = build_service(api_key)
    video_ID = get_id(video_url)
    
    comments_dict = comments_helper(video_ID, api_key, yt_service, filename = csv_filename)
    
    save_to_csv(comments_dict, csv_filename)
    
    print('Done.')
    

In [42]:
video_url = 'https://www.youtube.com/watch?v=_bwfAPXlFu8'
csv_filename = 'PM_camera_transitions'
api_key = 'creds.json'

get_comments(video_url, api_key, csv_filename = csv_filename)


Page no: 0
comment 0
comment 1
comment 2
comment 3
comment 4
comment 5
comment 6
comment 7
comment 8
comment 9
comment 10
comment 11
comment 12
comment 13
comment 14
comment 15
comment 16
comment 17
comment 18
comment 19
Page no: 1
comment 0
comment 1
comment 2
comment 3
comment 4
comment 5
comment 6
comment 7
comment 8
comment 9
comment 10
comment 11
comment 12
comment 13
comment 14
comment 15
comment 16
comment 17
comment 18
comment 19
Done.


In [43]:
comments_df = pd.read_csv('PM_camera_transitions.csv')
comments_df.head(15)

,Unnamed: 0,Comment,Author,Comment ID,Like Count,Viewer Rating
0,0,Hope you guys enjoy the 2MT! One of my favouri...,Peter McKinnon,UgySQDVAwHaeRKXpqQp4AaABAg,1058,none
1,1,What’s good people from 2021,Chris Lynch,UgwzfYceRAprxFuV-IN4AaABAg,0,none
2,2,Tip on taking good photos,Ezza,Ugzm28VWZWhedyYXvY54AaABAg,0,none
3,3,uploaded my first video and couple of days ago...,Abhisek Kishor,UgyzvSCX6oNVCoZDHDt4AaABAg,0,none
4,4,Really appreciate you as a teacher who is craz...,MakeYour VISH,UgwvQN3fMtqB3JzHdLV4AaABAg,0,none
5,5,Re-watch 3yrs latr!,BabyYoda,UgwUKN15n3icfMw5naV4AaABAg,0,none
6,6,oohhh Yeeeeeeeeees this is the show....thank y...,Riss PhotoVideo,UgwiHaIbsKysJGPqQsR4AaABAg,0,none
7,7,"Thanks, great video. I found the street footag...",Ollie Roberts,UgxDS1mCh6Q9CpBZ9xh4AaABAg,0,none
8,8,Honestly - I don't get it. I put the video in ...,Richard Krogh,UgxYBG55-cqERwjbGs14AaABAg,0,none
9,9,From Kenya💯🤣,Alvin Dowpy,UgwzB9PjMuxkf5FXP9l4AaABAg,0,none


In [44]:
comments_df.tail(10)

,Unnamed: 0,Comment,Author,Comment ID,Like Count,Viewer Rating
30,30,Thank you awesome video subscribed!,Rampage Rabbit,Ugx-_q9Iucil3h05GUR4AaABAg,0,none
31,31,Congrats for 5M subscribers 🔥🔥🔥,Eswar,UgxUxZN-NDyijkOr1SV4AaABAg,1,none
32,32,“2MT” \n*looks at the length of video*\n*says ...,Mark of God,Ugxcr-bzxd0Egxu8Cqh4AaABAg,0,none
33,33,Hi Peter Sanjay here Director and camera opera...,Sanjay Londhe,UgyBMuhHVAHVvzzN16J4AaABAg,0,none
34,34,Does Any knows which music is he using for the...,SOUL PHOTOGRAPHER,UgyoVcw8up6nmjEPp-B4AaABAg,0,none
35,35,Good day sir am new here can you tell how get ...,Guns and Rosary Man from the north,UgyOWYqTM9B-GOUDpsl4AaABAg,0,none
36,36,"""Smiling faces:\n\nMe: -_- lmao",D’Antoine LeVert,UgxDge0uGkaJsgTcrvR4AaABAg,0,none
37,37,whats the name of the song,Ji Im Jeong Sang Ho Byun &,UgxODglnnXqsVDoCYg14AaABAg,0,none
38,38,good tips but excuse me :D it would've be awes...,Ahmed mowafy,UgzoSLsV0ipa_nTtR2V4AaABAg,0,none
39,39,Your tutorials are awesome !!!!! Thank you,- Averumify -,Ugyk67HdFN1GvOPdhHF4AaABAg,0,none


In [48]:
comments_df.loc[38][1]

"good tips but excuse me :D it would've be awesum if u added how to edit/ speed it on premier pro.\nthank you and I learn a lot from you"

In [7]:
# run 2--removing unncessary columns
def comments_helper_2(video_ID, api_key, service, filename):
    comments, commentsId, likesCount, authors = [], [], [], []
    
    response = service.commentThreads().list(
        part="snippet", 
        videoId = video_ID, 
        textFormat="plainText").execute()
    
    page = 0
    
    while page < 2:
        print(f'Page no: {page}')
        page += 1
        index = 0
        
        for item in response['items']:
            print(f"comment {index}")
            
            index += 1

            comment = item["snippet"]["topLevelComment"]
            
            author = comment["snippet"]["authorDisplayName"]
            text = comment["snippet"]["textDisplay"]
            comment_id = item['snippet']['topLevelComment']['id']
            like_count = item['snippet']['topLevelComment']['snippet']['likeCount']

            # append to corresponding list
            comments.append(text)
            commentsId.append(comment_id)
            likesCount.append(like_count)
            authors.append(author)

#             with open(f'{csv_filename}.csv', 'a+') as f:
#                 csv_writer = writer(f)
#                 csv_writer.writerow([text, comment_id, like_count, author])

        
        # check if there's a next page
        if 'nextPageToken' in response:
            response = service.commentThreads().list(
                part="snippet", 
                videoId = video_ID, 
                textFormat="plainText",
                pageToken=response['nextPageToken']
            ).execute()
        else:
            break

    return {
        'Comment': comments,
        'Author' : authors,
        'Comment ID' : commentsId,
        'Like Count' : likesCount,
    }
    

In [8]:
def get_comments(video_url, api_key, order = 'time', 
                   part = 'snippet', maxResults = 100, 
                   csv_filename = 'video.csv' ):
    
    yt_service = build_service(api_key)
    video_ID = get_id(video_url)
    
    comments_dict = comments_helper_2(video_ID, api_key, yt_service, filename = csv_filename)
    
    save_to_csv(comments_dict, csv_filename)
    
    print('Done.')


In [9]:
video_url = 'https://www.youtube.com/watch?v=_bwfAPXlFu8'
csv_filename = 'PM_camera_transitions'
api_key = 'creds.json'

get_comments(video_url, api_key, csv_filename = csv_filename)

Page no: 0
comment 0
comment 1
comment 2
comment 3
comment 4
comment 5
comment 6
comment 7
comment 8
comment 9
comment 10
comment 11
comment 12
comment 13
comment 14
comment 15
comment 16
comment 17
comment 18
comment 19
Page no: 1
comment 0
comment 1
comment 2
comment 3
comment 4
comment 5
comment 6
comment 7
comment 8
comment 9
comment 10
comment 11
comment 12
comment 13
comment 14
comment 15
comment 16
comment 17
comment 18
comment 19
Done.


In [10]:
comments_df = pd.read_csv('PM_camera_transitions.csv')
comments_df.head(15)

,Unnamed: 0,Comment,Author,Comment ID,Like Count
0,0,Hope you guys enjoy the 2MT! One of my favouri...,Peter McKinnon,UgySQDVAwHaeRKXpqQp4AaABAg,1058
1,1,What’s good people from 2021,Chris Lynch,UgwzfYceRAprxFuV-IN4AaABAg,0
2,2,Tip on taking good photos,Ezza,Ugzm28VWZWhedyYXvY54AaABAg,0
3,3,uploaded my first video and couple of days ago...,Abhisek Kishor,UgyzvSCX6oNVCoZDHDt4AaABAg,0
4,4,Really appreciate you as a teacher who is craz...,MakeYour VISH,UgwvQN3fMtqB3JzHdLV4AaABAg,0
5,5,Re-watch 3yrs latr!,BabyYoda,UgwUKN15n3icfMw5naV4AaABAg,0
6,6,oohhh Yeeeeeeeeees this is the show....thank y...,Riss PhotoVideo,UgwiHaIbsKysJGPqQsR4AaABAg,0
7,7,"Thanks, great video. I found the street footag...",Ollie Roberts,UgxDS1mCh6Q9CpBZ9xh4AaABAg,0
8,8,Honestly - I don't get it. I put the video in ...,Richard Krogh,UgxYBG55-cqERwjbGs14AaABAg,0
9,9,From Kenya💯🤣,Alvin Dowpy,UgwzB9PjMuxkf5FXP9l4AaABAg,0


In [11]:
comments_df.sort_values('Like Count', ascending=False)

,Unnamed: 0,Comment,Author,Comment ID,Like Count
0,0,Hope you guys enjoy the 2MT! One of my favouri...,Peter McKinnon,UgySQDVAwHaeRKXpqQp4AaABAg,1058
17,17,WHAT IS THIS SONG!!! IT'S SO DOPE!,Ian Aitken,UgyCbL1jyqj3E1h2nwZ4AaABAg,1
31,31,Congrats for 5M subscribers 🔥🔥🔥,Eswar,UgxUxZN-NDyijkOr1SV4AaABAg,1
29,29,i can't believe this peter this is my country man,SamSai MediaZ,UgxZpHuqgRHxWGxlz_J4AaABAg,0
23,23,I'm here preparing for a shoot 5 hours from no...,The Name's OP,UgygWqqcPNuCFfXRuMh4AaABAg,0
24,24,Did you just say KENYA!,Pcc media,UgwrzV1oJ_tGsNaACAV4AaABAg,0
25,25,What a great video PETER! Wow again. Seriously...,Isabella Uribe,Ugy3z8eWl95TZJ3iVZF4AaABAg,0
26,26,Ayyyyy I thought I recognized Milwaukee. Come ...,Genelle Photography,UgxtjQQKDO0l6EuZc354AaABAg,0
27,27,Hi Peter...\nHow are you...\nI'm from Indonesi...,Tisu Putih,UgwGBKFlS-hmxwZIwXl4AaABAg,0
28,28,this is awesome video,Win's Life,Ugx_T_bP_Ap5NnnXWx94AaABAg,0
